# 0) (Optional) sanity: status + calendars

In [1]:
# Cell 0 — status + calendars
import requests

BASE = "http://127.0.0.1:8765"  # change if you run CalBridge on another port

print("STATUS:", requests.get(f"{BASE}/status", timeout=10).json())

print("\nCALENDARS:")
for c in requests.get(f"{BASE}/calendars", timeout=10).json():
    print(f"- {c['title']:<20} writable={c['allows_modifications']} id={c['id']}")


STATUS: {'authorized': True, 'status_code': 3}

CALENDARS:
- Home                 writable=True id=E198547B-614F-4109-B61A-96C56D4DED8A
- Birthdays            writable=False id=3CAFCE49-B7E6-4643-957A-5D2C095ABDCC
- US Holidays          writable=False id=BB8C7E8D-EA0D-43BC-B670-5940F97624F2
- Work                 writable=True id=20E9D11E-48F8-46E7-9CDB-73B49C70FD41


# 1) Create → verify → delete (default calendar)

In [2]:
# Cell 1A — create on DEFAULT calendar
import requests
from datetime import datetime, timedelta

BASE = "http://127.0.0.1:8765"

start = (datetime.now().astimezone() + timedelta(minutes=5)).isoformat()
end   = (datetime.now().astimezone() + timedelta(minutes=35)).isoformat()

resp = requests.post(f"{BASE}/add", json={
    "title": "Test: default calendar event",
    "start_iso": start,
    "end_iso": end,
    "notes": "source=notebook; exp=1A"
}, timeout=10)
print("CREATE status:", resp.status_code, resp.text)
exp1_event = resp.json().get("id")
print("EVENT_ID:", exp1_event)


CREATE status: 200 {"title":"Test: default calendar event","start_iso":"2025-10-03T16:33:18.222307-04:00","end_iso":"2025-10-03T17:03:18.222381-04:00","id":"EBD1F876-A2EE-4C2D-A829-71D5AB5FE60C","calendar":"Work"}
EVENT_ID: EBD1F876-A2EE-4C2D-A829-71D5AB5FE60C


In [3]:
# Cell 1B — delete the event from 1A
import requests

BASE = "http://127.0.0.1:8765"
assert 'exp1_event' in globals() and exp1_event, "Run Cell 1A first."

print("DELETE:", requests.post(f"{BASE}/delete", params={"event_id": exp1_event}, timeout=10).json())


DELETE: {'deleted': True}


# 2) Create with tags (title + notes) → verify → delete (default calendar)

In [6]:
# Cell 2A — create with tags on DEFAULT
import requests
from datetime import datetime, timedelta

BASE = "http://127.0.0.1:8765"

tags = ["work", "deepwork", "priority-high"]
TITLE = f"Notebook Tagged Event [{ ' '.join('#'+t for t in tags) }]"
NOTES = "tags: " + ",".join(tags) + "; source=notebook; exp=2A"

start = (datetime.now().astimezone() + timedelta(minutes=10)).isoformat()
end   = (datetime.now().astimezone() + timedelta(minutes=40)).isoformat()

resp = requests.post(f"{BASE}/add", json={
    "title": TITLE,
    "start_iso": start,
    "end_iso": end,
    "notes": NOTES
}, timeout=10)
print("CREATE status:", resp.status_code, resp.text)
exp2_event = resp.json().get("id")
print("EVENT_ID:", exp2_event)


CREATE status: 200 {"title":"Notebook Tagged Event [#work #deepwork #priority-high]","start_iso":"2025-10-03T16:40:06.784789-04:00","end_iso":"2025-10-03T17:10:06.784855-04:00","id":"8D7EC7E7-4AC4-4D23-8C9B-8C114E4A30BE","calendar":"Work"}
EVENT_ID: 8D7EC7E7-4AC4-4D23-8C9B-8C114E4A30BE


In [7]:
# Cell 2B — delete the tagged event from 2A
import requests

BASE = "http://127.0.0.1:8765"
assert 'exp2_event' in globals() and exp2_event, "Run Cell 2A first."

print("DELETE:", requests.post(f"{BASE}/delete", params={"event_id": exp2_event}, timeout=10).json())


DELETE: {'deleted': True}


# 3) Create in Work by calendar_id → verify → delete

In [8]:
# Cell 3A — create in WORK by ID
import requests
from datetime import datetime, timedelta

BASE = "http://127.0.0.1:8765"

# helper: get calendar id by exact title
def get_calendar_id_by_title(title: str):
    for c in requests.get(f"{BASE}/calendars", timeout=10).json():
        if (c["title"] or "").strip().lower() == title.strip().lower():
            return c["id"], c["allows_modifications"]
    return None, None

WORK_ID, writable = get_calendar_id_by_title("Work")
assert WORK_ID and writable, "Work calendar not found or not writable."

start = (datetime.now().astimezone() + timedelta(minutes=12)).isoformat()
end   = (datetime.now().astimezone() + timedelta(minutes=42)).isoformat()

resp = requests.post(f"{BASE}/add", json={
    "title": "Test: Work event (by ID)",
    "start_iso": start,
    "end_iso": end,
    "notes": "source=notebook; exp=3A",
    "calendar_id": WORK_ID
}, timeout=10)
print("CREATE status:", resp.status_code, resp.text)
exp3_event = resp.json().get("id")
print("EVENT_ID:", exp3_event)


CREATE status: 200 {"title":"Test: Work event (by ID)","start_iso":"2025-10-03T16:42:45.302802-04:00","end_iso":"2025-10-03T17:12:45.302853-04:00","id":"7BF57F70-2019-44FF-AE3F-89C191847F77","calendar":"Work"}
EVENT_ID: 7BF57F70-2019-44FF-AE3F-89C191847F77


In [9]:
# Cell 3B — delete the Work event from 3A
import requests

BASE = "http://127.0.0.1:8765"
assert 'exp3_event' in globals() and exp3_event, "Run Cell 3A first."

print("DELETE:", requests.post(f"{BASE}/delete", params={"event_id": exp3_event}, timeout=10).json())


DELETE: {'deleted': True}


# 4) Create in Home by calendar_title (fallback) → verify → delete

In [10]:
# Cell 4A — create in HOME by title (fallback)
import requests
from datetime import datetime, timedelta

BASE = "http://127.0.0.1:8765"

start = (datetime.now().astimezone() + timedelta(minutes=15)).isoformat()
end   = (datetime.now().astimezone() + timedelta(minutes=45)).isoformat()

resp = requests.post(f"{BASE}/add", json={
    "title": "Test: Home event (by title)",
    "start_iso": start,
    "end_iso": end,
    "notes": "source=notebook; exp=4A",
    "calendar_title": "Home"   # case-insensitive match
}, timeout=10)
print("CREATE status:", resp.status_code, resp.text)
exp4_event = resp.json().get("id")
print("EVENT_ID:", exp4_event)


CREATE status: 200 {"title":"Test: Home event (by title)","start_iso":"2025-10-03T16:46:35.423978-04:00","end_iso":"2025-10-03T17:16:35.424034-04:00","id":"60C8DC1D-513F-42D6-A469-D0BB6457F543","calendar":"Home"}
EVENT_ID: 60C8DC1D-513F-42D6-A469-D0BB6457F543


In [11]:
# Cell 4B — delete the Home event from 4A
import requests

BASE = "http://127.0.0.1:8765"
assert 'exp4_event' in globals() and exp4_event, "Run Cell 4A first."

print("DELETE:", requests.post(f"{BASE}/delete", params={"event_id": exp4_event}, timeout=10).json())


DELETE: {'deleted': True}


# 5) List next 7 days, grouped by calendar (read-only)

In [12]:
# Cell 5 — list next 7 days grouped by calendar
import requests, collections
from datetime import datetime

BASE = "http://127.0.0.1:8765"

events = requests.get(f"{BASE}/events", params={"days": 7}, timeout=15).json()
by_cal = collections.defaultdict(list)
for e in events:
    by_cal[e.get("calendar","(unknown)")].append(e)

for cal_name in sorted(by_cal.keys()):
    print(f"\n=== {cal_name} ===")
    for e in sorted(by_cal[cal_name], key=lambda x: x["start_iso"]):
        s = datetime.fromisoformat(e["start_iso"]).strftime("%Y-%m-%d %H:%M")
        en = datetime.fromisoformat(e["end_iso"]).strftime("%H:%M")
        print(f"- {s} → {en} | {e['title']} (id: {e['id'][:24]})")



=== Home ===
- 2025-10-05 10:00 → 10:30 | Test: Home event (id: 77857C54-5B3A-4F47-8DFC-)
- 2025-10-06 00:00 → 01:00 | TA Office Hours (id: AC7A3416-9489-4CC0-B959-)
- 2025-10-07 16:10 → 18:00 | TA: Practicum (id: 78321BA2-61E0-4B90-9BBC-)

=== Work ===
- 2025-10-04 10:00 → 10:30 | Test: Work event (id: 3C20A137-D715-470F-94D7-)


# 6) List next 30 days excluding holidays (read-only)

In [13]:
# Cell 6 — list next 30 days excluding holidays
import requests
from datetime import datetime

BASE = "http://127.0.0.1:8765"

events_30 = requests.get(f"{BASE}/events", params={"days": 30}, timeout=20).json()

def is_holiday(e):
    return "holiday" in (e.get("calendar") or "").lower()

filtered = [e for e in events_30 if not is_holiday(e)]

print(f"Found {len(filtered)} non-holiday events in next 30 days\n")
print(f"{'Start':<20} {'End':<20} {'Calendar':<14} Title")
print("-"*90)
for e in sorted(filtered, key=lambda x: x["start_iso"]):
    s = datetime.fromisoformat(e["start_iso"]).strftime("%Y-%m-%d %H:%M")
    en = datetime.fromisoformat(e["end_iso"]).strftime("%Y-%m-%d %H:%M")
    print(f"{s:<20} {en:<20} {str(e.get('calendar') or ''):<14} {e['title']}")


Found 10 non-holiday events in next 30 days

Start                End                  Calendar       Title
------------------------------------------------------------------------------------------
2025-10-04 10:00     2025-10-04 10:30     Work           Test: Work event
2025-10-05 10:00     2025-10-05 10:30     Home           Test: Home event
2025-10-06 00:00     2025-10-06 01:00     Home           TA Office Hours
2025-10-07 16:10     2025-10-07 18:00     Home           TA: Practicum
2025-10-13 00:00     2025-10-13 01:00     Home           TA Office Hours
2025-10-14 16:10     2025-10-14 18:00     Home           TA: Practicum
2025-10-20 00:00     2025-10-20 01:00     Home           TA Office Hours
2025-10-21 16:10     2025-10-21 18:00     Home           TA: Practicum
2025-10-27 00:00     2025-10-27 01:00     Home           TA Office Hours
2025-10-28 16:10     2025-10-28 18:00     Home           TA: Practicum


# 7) Negative test — try creating in a read-only calendar (should 4xx)

In [14]:
# Cell 7 — attempt to create in a read-only calendar (e.g., "US Holidays")
import requests
from datetime import datetime, timedelta

BASE = "http://127.0.0.1:8765"

# find a non-writable calendar id by title (e.g., "US Holidays")
def get_calendar(title: str):
    for c in requests.get(f"{BASE}/calendars", timeout=10).json():
        if (c["title"] or "").lower() == title.lower():
            return c["id"], c["allows_modifications"]
    return None, None

hol_id, writable = get_calendar("US Holidays")
if not hol_id:
    print("No 'US Holidays' calendar found on this system — skipping test.")
else:
    print("US Holidays writable?", writable)
    start = (datetime.now().astimezone() + timedelta(minutes=20)).isoformat()
    end   = (datetime.now().astimezone() + timedelta(minutes=50)).isoformat()
    resp = requests.post(f"{BASE}/add", json={
        "title": "Should fail: read-only calendar",
        "start_iso": start,
        "end_iso": end,
        "calendar_id": hol_id
    }, timeout=10)
    print("CREATE status:", resp.status_code)
    print("Body:", resp.text)


US Holidays writable? False
CREATE status: 400
Body: {"detail":"calendar_id not writable: BB8C7E8D-EA0D-43BC-B670-5940F97624F2 (US Holidays)"}


# 8) Create another tagged event, verify, then delete (explicit calendar_id)

In [15]:
# Cell 8A — create tagged event in WORK by ID
import requests
from datetime import datetime, timedelta

BASE = "http://127.0.0.1:8765"

# fetch WORK id
def get_calendar_id_by_title(title: str):
    for c in requests.get(f"{BASE}/calendars", timeout=10).json():
        if (c["title"] or "").strip().lower() == title.strip().lower():
            return c["id"], c["allows_modifications"]
    return None, None

WORK_ID, writable = get_calendar_id_by_title("Work")
assert WORK_ID and writable, "Work calendar not found or not writable."

tags = ["deepwork","proposal"]
TITLE = f"Exp8 Tagged [#"+ " #".join(tags) + "]"
NOTES = "tags:" + ",".join(tags) + "; source=notebook; exp=8A"

start = (datetime.now().astimezone() + timedelta(minutes=25)).isoformat()
end   = (datetime.now().astimezone() + timedelta(minutes=55)).isoformat()

resp = requests.post(f"{BASE}/add", json={
    "title": TITLE,
    "start_iso": start,
    "end_iso": end,
    "notes": NOTES,
    "calendar_id": WORK_ID
}, timeout=10)
print("CREATE status:", resp.status_code, resp.text)
exp8_event = resp.json().get("id")
print("EVENT_ID:", exp8_event)

# quick verify it appears in the next 3 days
evs = requests.get(f"{BASE}/events", params={"days": 3}, timeout=10).json()
print("FOUND:", any(e.get("id") == exp8_event for e in evs))


CREATE status: 200 {"title":"Exp8 Tagged [#deepwork #proposal]","start_iso":"2025-10-03T17:03:12.320656-04:00","end_iso":"2025-10-03T17:33:12.320703-04:00","id":"EE916233-9B0D-47E2-A28E-2F25E6CF2BCA","calendar":"Work"}
EVENT_ID: EE916233-9B0D-47E2-A28E-2F25E6CF2BCA
FOUND: True


In [16]:
# Cell 8B — delete event from 8A
import requests

BASE = "http://127.0.0.1:8765"
assert 'exp8_event' in globals() and exp8_event, "Run Cell 8A first."

print("DELETE:", requests.post(f"{BASE}/delete", params={"event_id": exp8_event}, timeout=10).json())


DELETE: {'deleted': True}
